# RAG GenAI demo using Mistral 7b (local LLM)
This requires a fairly hefty instance backing the notebook, >40GB RAM at least, if using CPU.
We used an ml.m5.4xlarge (no GPU) for the purposes of this demo, which is slow, but functional.

### Tutorial Outline
0. `pip install` and import relevant dependencies.
1. Download a public-facing pdf 
2. Split the document into chunks
3. Index the embeddings with FAISS and `all-MiniLM-l6-v2`.
4. Initialize our question, prompt and context (from FAISS similarity search)
5. Load the LLM (mistral 7b instruct)
6. Generate the output of the prompt and display

# 0. `pip install` and import relevant dependencies

In [2]:
pip install faiss-cpu PyPDF2 langchain sentence-transformers transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 41.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.0 MB/s eta 0:00:00ta 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 1.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.6 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 1.6 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 40.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 11.0 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 48.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 9.5 MB/s

In [3]:
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
from transformers import AutoTokenizer, AutoModelForCausalLM
from IPython.display import display, Markdown

# 1. Download a public-facing pdf

In [4]:
%%sh
wget -O fannie-mf-commentary-oct-2023.pdf https://www.fanniemae.com/media/49331/display

--2024-01-10 23:33:12--  https://www.fanniemae.com/media/49331/display
Resolving www.fanniemae.com (www.fanniemae.com)... 104.18.26.25, 104.18.27.25, 2606:4700::6812:1a19, ...
Connecting to www.fanniemae.com (www.fanniemae.com)|104.18.26.25|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 249442 (244K) [application/pdf]
Saving to: ‘fannie-mf-commentary-oct-2023.pdf’

     0K .......... .......... .......... .......... .......... 20% 43.6M 0s
    50K .......... .......... .......... .......... .......... 41%  186M 0s
   100K .......... .......... .......... .......... .......... 61%  241M 0s
   150K .......... .......... .......... .......... .......... 82% 93.9M 0s
   200K .......... .......... .......... .......... ...       100%  227M=0.002s

2024-01-10 23:33:12 (104 MB/s) - ‘fannie-mf-commentary-oct-2023.pdf’ saved [249442/249442]



# 2. Split the document into chunks

In [5]:
# split up our document into chunks
docs = []
text_splitter = RecursiveCharacterTextSplitter(chunk_size=950, chunk_overlap=150)
filename = 'fannie-mf-commentary-oct-2023.pdf'
reader = PdfReader(filename)

for idx, page in enumerate(reader.pages):
    text = page.extract_text()
    if len(text) > 0:
        docs.extend(text_splitter.create_documents(texts=[text],metadatas=[{'filename': filename, 'page': idx+1}]))

# show what we got from this code
docs[0:1]

[Document(page_content='1Multifamily Economic and Market Commentary\nOCTOBER 2023\nRising Number of Multifamily Properties Offering Concessions\nMultifamily market fundamentals have softened in 2023 compared to the prior year, the result of mixed economic \ntrends including slowing -but-still -positive job growth, elevated single -family housing prices keeping many renters in \nplace, and continued favorable demographics. Rent growth was exceptional over the past two years, and, of course, \nunsustainable, thus 2023 has seen a substantial slowing of rent growth rates. There remains a robust pipeline of new \napartment rental projects that are underway in the nation’s largest metros, and with recessionary concerns there has \nbeen a rise in the number of properties across the country offering concessions.\nIn the multifamily apartment rental market, concessions are incentives with an economic value for renters, such as', metadata={'filename': 'fannie-mf-commentary-oct-2023.pdf', 'page':

# 3. Index the embeddings with FAISS and `all-MiniLM-l6-v2`

In [6]:
embeddings = HuggingFaceEmbeddings(
    model_name='sentence-transformers/all-MiniLM-l6-v2',
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': False}
)

db = FAISS.from_documents(docs, embeddings)

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

# 4. Initialize our question, prompt, and context (from FAISS similarity search)

In [7]:
# something like: "what is the multifamily market forecast for 2024?"
# or, "what concerns are there going forward for the apartment industry?"
prompt = input()

 what concerns are there going forward for the apartment industry?


In [8]:
search_docs = db.similarity_search(prompt)
# get the top 3 search results
top_search_docs = search_docs[0:3]
top_search_docs

[Document(page_content='1Multifamily Economic and Market Commentary\nOCTOBER 2023\nRising Number of Multifamily Properties Offering Concessions\nMultifamily market fundamentals have softened in 2023 compared to the prior year, the result of mixed economic \ntrends including slowing -but-still -positive job growth, elevated single -family housing prices keeping many renters in \nplace, and continued favorable demographics. Rent growth was exceptional over the past two years, and, of course, \nunsustainable, thus 2023 has seen a substantial slowing of rent growth rates. There remains a robust pipeline of new \napartment rental projects that are underway in the nation’s largest metros, and with recessionary concerns there has \nbeen a rise in the number of properties across the country offering concessions.\nIn the multifamily apartment rental market, concessions are incentives with an economic value for renters, such as', metadata={'filename': 'fannie-mf-commentary-oct-2023.pdf', 'page':

In [9]:
context = " ----- ".join([search_doc.page_content for search_doc in top_search_docs])
prompt_template = f"""[INST] You are a helpful AI assistant. Use the following pieces of context to answer the question at the end. 
 If you can't find the answer from the context, just say that you don't know.
 ----- 
Context: 
 {context}
 ----- 
Question: {prompt}[/INST]"""
prompt_template

"[INST] You are a helpful AI assistant. Use the following pieces of context to answer the question at the end. \n If you can't find the answer from the context, just say that you don't know.\n ----- \nContext: \n 1Multifamily Economic and Market Commentary\nOCTOBER 2023\nRising Number of Multifamily Properties Offering Concessions\nMultifamily market fundamentals have softened in 2023 compared to the prior year, the result of mixed economic \ntrends including slowing -but-still -positive job growth, elevated single -family housing prices keeping many renters in \nplace, and continued favorable demographics. Rent growth was exceptional over the past two years, and, of course, \nunsustainable, thus 2023 has seen a substantial slowing of rent growth rates. There remains a robust pipeline of new \napartment rental projects that are underway in the nation’s largest metros, and with recessionary concerns there has \nbeen a rise in the number of properties across the country offering concessi

# 5. Load the LLM (mistral 7b instruct)

In [10]:
model_name = 'mistralai/Mistral-7B-Instruct-v0.1'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

# 6. Generate the output of the prompt and display

In [11]:
# tokenize 
input_ids = tokenizer.encode(prompt_template, return_tensors='pt')
output = model.generate(
    input_ids,
    max_new_tokens=500,
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id,
)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

# note that this step takes several minutes on a CPU instance

In [12]:
# convert the similarity search results to markdown to make it pretty
md_search_docs = ''
for doc in top_search_docs:
    md_search_docs += f"### File: {doc.metadata['filename']}, Page: {doc.metadata['page']}\n"
    md_search_docs += doc.page_content + "\n"

# combine the model generated output and the context
result = f'''## Question: 
{prompt}

## Answer: 
{generated_text[generated_text.find("[/INST]")+7:]}

## Context used: 
{md_search_docs}
'''
display(Markdown(result))

## Question: 
what concerns are there going forward for the apartment industry?

## Answer: 
 According to the multifamily economic and market commentary, there are concerns that the forecasted economic slowdown, combined with the robust supply of new apartment units that are underway, will add stress to the multifamily sector over the next 12 to 18 months. There is also a possibility that further rise in the number of units offering concessions may be on the horizon due to the weakened demand from the first half of 2023, resulting in stagnant rent growth and higher vacancy rates.

## Context used: 
### File: fannie-mf-commentary-oct-2023.pdf, Page: 1
1Multifamily Economic and Market Commentary
OCTOBER 2023
Rising Number of Multifamily Properties Offering Concessions
Multifamily market fundamentals have softened in 2023 compared to the prior year, the result of mixed economic 
trends including slowing -but-still -positive job growth, elevated single -family housing prices keeping many renters in 
place, and continued favorable demographics. Rent growth was exceptional over the past two years, and, of course, 
unsustainable, thus 2023 has seen a substantial slowing of rent growth rates. There remains a robust pipeline of new 
apartment rental projects that are underway in the nation’s largest metros, and with recessionary concerns there has 
been a rise in the number of properties across the country offering concessions.
In the multifamily apartment rental market, concessions are incentives with an economic value for renters, such as
### File: fannie-mf-commentary-oct-2023.pdf, Page: 4
new ones. And that would still include all classes of units, including the more affordable Class B and C.
We believe the forecasted economic slowdown, combined with the robust supply of new apartment units that 
are underway, will likely add stress to the multifamily sector over the next 12 to 18 months. But we continue to 
believe that demand for multifamily rental housing will remain stable over the longer term due to home 
purchase affordability constraints and higher interest rates keeping many tenants from moving to 
homeownership. As a result, we believe that many renters -by-choice will stay in their units for slightly longer, 
especially if they are able to take advantage of lower rent levels and more generous concessions.
### File: fannie-mf-commentary-oct-2023.pdf, Page: 4
4Multifamily Economic and Market Commentary
Further Softening Expected to Lead to More Concessions and Stagnant Rent Growth
After exceptional rent growth rates in 2021 and 2022, it was generally expected that 2023 would see easing of 
multifamily rent growth and overall fundamentals, including vacancy rates. In general, that has been the case. 
Softening in demand during the first half of this year resulted in a below -average rent growth of just 1.0% for 
the nation, and a modest rise in vacancies. But some metros had notably weaker results, experiencing modest 
rent contractions, including Austin, Las Vegas, San Francisco, and Phoenix. 
Because of this fundamental demand softness, and our current expectation of a mild recession in the first half 
of 2024, a further rise in the number of units offering concessions may be on the horizon. It is noteworthy that

